# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Check current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the cvs data file that will be used for Apache Cassandra tables

In [3]:
# Initiate an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:
    # Read csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Create a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
       
        # Extract each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Get total number of rows 
print('Total number of rows:', len(full_data_rows_list))

# Create a smaller event_datafile_full csv file that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total number of rows: 8056


#### Checking event_datefile_new.csv file

In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Number of rows:', sum(1 for line in f))

# Check what data in event_datafile_new.csv looks like
data = pd.read_csv('event_datafile_new.csv', encoding='utf-8', index_col=False)
data.head()

Number of rows: 6821


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance on local machine 
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_app_data
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
"""
)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('music_app_data')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """
    CREATE TABLE IF NOT EXISTS song_in_session (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession))
"""

session.execute(query)                

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO song_in_session (
                sessionId,
                itemInSession,
                artist,
                song,
                length)
            VALUES (%s, %s, %s, %s, %s)
        """
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT * FROM song_in_session
    WHERE sessionId = 338 AND itemInSession = 4 
"""
df = []
row = session.execute(query)[0]
df.append([row.sessionid, row.iteminsession, row.artist, row.song, row.length])

column_names = ['session_id', 'item_in_session', 'artist', 'song', 'length']

df = pd.DataFrame (df, columns = column_names)
df

,session_id,item_in_session,artist,song,length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.307312


In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = """
    CREATE TABLE IF NOT EXISTS user_songs (
        userId int,
        sessionId int,
        itemInSession int,
        song text, 
        artist text, 
        firstName text,
        lastName text,
    PRIMARY KEY (userId, sessionId, itemInSession))
"""

session.execute(query)    
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO user_songs (
                userId,
                sessionId,
                itemInSession,
                song, 
                artist, 
                firstName,
                lastName)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4] ))

In [13]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT * FROM user_songs
    WHERE userId = 10 AND sessionId = 182
"""
df = []
rows = session.execute(query)

for row in rows:
    df.append([row.userid, row.sessionid, row.iteminsession, row.artist, row.song, row.firstname, row.lastname])

column_names = ['user_id', 'session_id', 'item_in_session', 'artist', 'song', 'firstname', 'lastname']

df = pd.DataFrame (df, columns = column_names)
df

,user_id,session_id,item_in_session,artist,song,firstname,lastname
0,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,10,182,1,Three Drives,Greece 2000,Sylvie,Cruz
2,10,182,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
    CREATE TABLE IF NOT EXISTS song_listeners (
        song text, 
        userId int, 
        firstName text, 
        lastName text,
        PRIMARY KEY (song, userId))"""

session.execute(query)

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """
            INSERT INTO song_listeners (
                song,
                userId,
                firstName,
                lastName)
            VALUES (%s, %s, %s, %s)
        """
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT firstName, lastName FROM song_listeners
    WHERE song = 'All Hands Against His Own'
"""
df = []
rows = session.execute(query)

for row in rows:
    df.append([row.firstname, row.lastname])

column_names = ['first_name', 'last_name']

df = pd.DataFrame (df, columns = column_names)
df

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions

In [18]:
query = "drop table song_in_session"
session.execute(query)

query = "drop table user_songs"
session.execute(query)

query = "drop table song_listeners"
session.execute(query)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()